In [151]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv(r'/Users/sehan/Documents/Dataset/Multi_New/top_15_multi.csv')
df.head()

,Load,SrcTCPBase,tcp,udp,RST,INT,sHops,dTtl,e s,Loss,SrcLoss,AckDat,TcpRtt,FIN,IdleTime,Attack Type
0,0.0,NaN,0,1,0,0,0.0,63.0,0,0,0,0.0,0.0,0,1655716224,0
1,0.0,NaN,0,1,0,0,0.0,63.0,0,0,0,0.0,0.0,0,1655716224,0
2,0.0,NaN,0,1,0,0,0.0,63.0,0,0,0,0.0,0.0,0,1655716224,0
3,0.0,NaN,0,1,0,0,0.0,63.0,0,0,0,0.0,0.0,0,1655716224,0
4,0.0,NaN,0,0,0,0,6.0,64.0,0,0,0,0.0,0.0,0,1655716224,0


In [152]:
df = df.fillna(df.median())
print(df)

                Load    SrcTCPBase  tcp  udp  RST  INT  sHops   dTtl  \
0       0.000000e+00  1.490814e+09    0    1    0    0    0.0   63.0   
1       0.000000e+00  1.490814e+09    0    1    0    0    0.0   63.0   
2       0.000000e+00  1.490814e+09    0    1    0    0    0.0   63.0   
3       0.000000e+00  1.490814e+09    0    1    0    0    0.0   63.0   
4       0.000000e+00  1.490814e+09    0    0    0    0    6.0   64.0   
...              ...           ...  ...  ...  ...  ...    ...    ...   
851199  8.619034e+06  1.490814e+09    0    1    0    0   11.0   64.0   
851200  3.068197e+06  1.490814e+09    0    1    0    0   11.0   64.0   
851201  3.064397e+04  1.490814e+09    0    1    0    0    0.0  117.0   
851202  3.179072e+04  1.490814e+09    0    1    0    0    0.0  117.0   
851203  5.227891e+05  1.490814e+09    0    1    0    0   11.0   64.0   

         e s        Loss  SrcLoss  AckDat  TcpRtt  FIN    IdleTime  \
0                0     0        0     0.0     0.0    0  165571622

In [153]:
X = df.iloc[:,0:14].values
y = df.iloc[:,15].values

In [154]:
#test and train split (30% for test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=0)

In [155]:
#data normalization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [156]:
import time
from sklearn import tree 
model = tree.DecisionTreeClassifier()
start = time.time()
model.fit(X_train, y_train)
stop = time.time()
print(f"Training time: {stop - start}s")

Training time: 4.671222925186157s


In [158]:
from sklearn.metrics import roc_auc_score
start = time.time()
y_pred = model.predict_proba(X_test)
stop = time.time()
y_pred
print(f"Prediction time: {stop - start}s")

Prediction time: 0.07710099220275879s


In [150]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))
print(accuracy_score(y_test, y_pred))

[[ 9844   120     9     0     0     4     1]
 [    5  9692     0     0     0     2     0]
 [    0     0 67352     0     0     0     0]
 [    0   528     0 79323     0     0     0]
 [ 2841     0     0     0 43927     0     0]
 [   18     0     0     0     0 28664     0]
 [   16     0     0     0     0     0 13016]]
              precision    recall  f1-score   support

           0     0.7737    0.9866    0.8672      9978
           1     0.9373    0.9993    0.9673      9699
           2     0.9999    1.0000    0.9999     67352
           3     1.0000    0.9934    0.9967     79851
           4     1.0000    0.9393    0.9687     46768
           5     0.9998    0.9994    0.9996     28682
           6     0.9999    0.9988    0.9993     13032

    accuracy                         0.9861    255362
   macro avg     0.9587    0.9881    0.9713    255362
weighted avg     0.9887    0.9861    0.9867    255362

0.9861216625809635


In [159]:
from sklearn.metrics import roc_curve, auc

model_fpr, model_tpr, threshold = roc_curve(y_test, y_pred[:,1])
auc_model = auc(model_fpr, model_tpr)

plt.figure(figsize=(5, 5), dpi=100)
plt.plot(model_fpr, model_tpr, linestyle='-', label='DT (auc = %0.8f)' % auc_model)
plt.plot([0, 1], [0, 1], color="darkorange", lw=2, linestyle="--")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.legend()
plt.show()

ValueError: multiclass format is not supported